In [ ]:
import os
from itertools import chain
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import utils
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, GlobalAveragePooling2D, LSTM, TimeDistributed, Dropout, Dense, BatchNormalization, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator, array_to_img
import numpy as np
from PIL import Image
from tqdm import tqdm
from tqdm.keras import TqdmCallback
import pandas as pd
import math
import matplotlib.pyplot as plt
import shutil
from collections import defaultdict
from sklearn.model_selection import train_test_split
import pandas as pd
from typing import List

from data import Data
from metrics import SlicewiseAccuracy
from data_generator import PhaseDataGenerator, SliceDataGenerator
from results import *

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
keras_app = tf.keras.applications.mobilenet
keras_model = tf.keras.applications.mobilenet.MobileNet

## Initializer Data Pipeline

In [ ]:
data = Data()

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=keras_app.preprocess_input)
phase_gen = PhaseDataGenerator(data, "KAG", target_size=(224, 224), batch_size=2,
                               shuffle=True, image_data_generator=datagen)
slice_gen = SliceDataGenerator(data, "KAG", target_size=(224, 224), batch_size=32,
                               shuffle=True, image_data_generator=datagen)

In [ ]:
phase_gen.get_slice_index()

In [ ]:
assert(np.isclose(slice_gen[0][0][0].min(), -1, rtol=1.0e-4))
assert(np.isclose(slice_gen[0][0][0].max(), 1, rtol=1.0e-4))

In [ ]:
assert(np.isclose(phase_gen[0][0][0].min(), -1, rtol=1.0e-4))
assert(np.isclose(phase_gen[0][0][0].max(), 1, rtol=1.0e-4))

In [ ]:
backbone = keras_model(include_top=False, pooling='avg', weights='imagenet', input_shape=(224, 224, 3))
backbone.trainable = False

## Intermediate Slice Extraction
Skip this part for end-to-end model training

In [ ]:
# Extractors
cnn_ext = backbone
rnn_ext = Sequential()
rnn_ext.add(TimeDistributed(backbone))

In [ ]:
x_cnn = cnn_ext.predict(slice_test_gen, verbose=1)

In [ ]:
x_rnn = rnn_ext.predict(phase_test_gen, verbose=1)

In [ ]:
x_cnn.shape

In [ ]:
y_cnn = np.ndarray((0, 5))
for _, labels in tqdm(slice_test_gen):
    y_cnn = np.concatenate([y_cnn, labels])

In [ ]:
y_rnn = np.ndarray((0, 25, 5))
for _, labels in tqdm(phase_test_gen):
    y_rnn = np.concatenate([y_rnn, labels])

### Data Split

In [ ]:
patients = data.data["KAG"]

In [ ]:
patient_ids = np.array(list(patients.keys()))
slices_by_patient = defaultdict(int)
slice_ids_by_patient = dict()
phase_ids_by_patient = dict()

In [ ]:
train_pids, test_pids = train_test_split(patient_ids, train_size=0.8, random_state=0)
train_pids.sort()
test_pids.sort()

In [ ]:
current = 0
for i, (patient, phases) in enumerate(sorted(list(patients.items()))):
    phases = list(phases.values())
    n_slices = len(phases[0])
    slices_by_patient[patient] = n_slices
    slice_ids = list(range(current, current + n_slices * 2))
    slice_ids_by_patient[patient] = slice_ids
    phase_ids_by_patient[patient] = [i * 2, i * 2 + 1]
    current = current + n_slices * 2
    assert(len(phases[0]) == len(phases[1]))

In [ ]:
slice_train_sids = []
slice_test_sids = []
for pid in train_pids:
    slice_train_sids.extend(slice_ids_by_patient[pid])
for pid in test_pids:
    slice_test_sids.extend(slice_ids_by_patient[pid])
    
phase_train_sids = []
phase_test_sids = []
for pid in train_pids:
    phase_train_sids.extend(phase_ids_by_patient[pid])
for pid in test_pids:
    phase_test_sids.extend(phase_ids_by_patient[pid])

In [ ]:
x_cnn_train = x_cnn[slice_train_sids]
x_cnn_test = x_cnn[slice_test_sids]
y_cnn_train = y_cnn[slice_train_sids]
y_cnn_test = y_cnn[slice_test_sids]

x_rnn_train = x_rnn[phase_train_sids]
x_rnn_test = x_rnn[phase_test_sids]
y_rnn_train = y_rnn[phase_train_sids]
y_rnn_test = y_rnn[phase_test_sids]

In [ ]:
rnn_top = Sequential()
rnn_top.add(LSTM(256, input_shape=(25, 1024), return_sequences=True))
rnn_top.add(Dropout(0.5))
#rnn_top.add(TimeDistributed(Dense(256, activation="relu")))
#rnn_top.add(TimeDistributed(Dropout(0.5)))
rnn_top.add(TimeDistributed(Dense(5, activation="softmax")))

In [ ]:
rnn_top.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=[SlicewiseAccuracy()])

In [ ]:
acc = list()
loss = list()
vacc = list()
vloss = list()
for i in range(5):
    print("Iteration", i)
    rnn_top = Sequential()
    rnn_top.add(LSTM(256, input_shape=(25, 1024), return_sequences=True))
    rnn_top.add(Dropout(0.5))
    #rnn_top.add(TimeDistributed(Dense(256, activation="relu")))
    #rnn_top.add(TimeDistributed(Dropout(0.5)))
    rnn_top.add(TimeDistributed(Dense(5, activation="softmax")))
    rnn_top.compile(loss="categorical_crossentropy",
                    optimizer="adam",
                    metrics=[SlicewiseAccuracy()])
    history = rnn_top.fit(x_rnn_train, y_rnn_train, validation_data=(x_rnn_test, y_rnn_test),
                          batch_size=2, epochs=100, verbose=0)
    acc.append(np.array(history.history["slicewise_accuracy"]))
    loss.append(np.array(history.history["loss"]))
    vacc.append(np.array(history.history["val_slicewise_accuracy"]))
    vloss.append(np.array(history.history["val_loss"]))

In [ ]:
rhistory = dict(
    accuracy=np.stack(acc).mean(axis=0),
    loss=np.stack(loss).mean(axis=0),
    val_accuracy=np.stack(vacc).mean(axis=0),
    val_loss=np.stack(vloss).mean(axis=0),
)

In [ ]:
cnn_top = Sequential()
cnn_top.add(Dense(256, activation="relu"))
cnn_top.add(Dropout(0.5))
cnn_top.add(Dense(5, activation="softmax"))

In [ ]:
cnn_top.compile(loss="categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [ ]:
cacc = list()
closs = list()
cvacc = list()
cvloss = list()
for i in range(5):
    print("Iteration", i)
    cnn_top = Sequential()
    cnn_top.add(Dense(256, activation="relu"))
    cnn_top.add(Dropout(0.5))
    cnn_top.add(Dense(5, activation="softmax"))
    cnn_top.compile(loss="categorical_crossentropy",
                    optimizer="adam",
                    metrics=["accuracy"])
    history = cnn_top.fit(x_cnn_train, y_cnn_train, validation_data=(x_cnn_test, y_cnn_test),
                          batch_size=32, epochs=100, verbose=0)
    cacc.append(np.array(history.history["accuracy"]))
    closs.append(np.array(history.history["loss"]))
    cvacc.append(np.array(history.history["val_accuracy"]))
    cvloss.append(np.array(history.history["val_loss"]))

In [ ]:
chistory = dict(
    accuracy=np.stack(cacc).mean(axis=0),
    loss=np.stack(closs).mean(axis=0),
    val_accuracy=np.stack(cvacc).mean(axis=0),
    val_loss=np.stack(cvloss).mean(axis=0),
)

In [ ]:
fig = plt.Figure(figsize=(10, 6))
fig.suptitle("CNN=dotted, RNN=solid")
ax = fig.subplots()
pd.DataFrame(rhistory).plot(ylim=[0, 1], ax=ax)
ax.set_prop_cycle(None)
pd.DataFrame(chistory).plot(ylim=[0, 1], ax=ax, linestyle="dotted", linewidth=2)
fig

In [ ]:
cnn_top.evaluate(x_cnn_test, y_cnn_test)

In [ ]:
rnn_top.evaluate(x_rnn_test, y_rnn_test)

## End-To-End Models

In [ ]:
cnn_model = Sequential()
cnn_model.add(backbone)
cnn_model.add(Dense(256, activation="relu"))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(5, activation="softmax"))

cnn_model.layers[0].trainable = False
cnn_model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [ ]:
cnn_history = cnn_model.fit(slice_gen, epochs=1)

In [ ]:
rnn_model = Sequential()
rnn_model.add(TimeDistributed(backbone))
rnn_model.add(LSTM(256, input_shape=(25, 2048), return_sequences=True))
rnn_model.add(Dropout(0.5))
#rnn_model.add(TimeDistributed(Dense(256, activation="relu")))
#rnn_model.add(TimeDistributed(Dropout(0.5)))
rnn_model.add(TimeDistributed(Dense(5, activation="softmax")))

In [ ]:
rnn_model.layers[0].trainable = False
rnn_model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=[SlicewiseAccuracy()])

In [ ]:
rnn_history = rnn_model.fit(phase_gen, epochs=1)

## Evaluate

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=keras_app.preprocess_input)
cnn_testgen = SliceDataGenerator(data, "KAG", target_size=(224, 224), batch_size=32,
                                 shuffle=False, image_data_generator=datagen)
rnn_testgen = PhaseDataGenerator(data, "KAG", target_size=(224, 224), batch_size=2,
                                 shuffle=False, image_data_generator=datagen)

In [ ]:
cnn_model.evaluate(cnn_testgen)

In [ ]:
rnn_model.evaluate(rnn_testgen)

In [ ]:
cnn_preds = cnn_model.predict(cnn_testgen)

In [ ]:
rnn_preds = rnn_model.predict(rnn_testgen)

In [ ]:
cnn_preds.shape, rnn_preds.shape

In [ ]:
slice_index = rnn_testgen.get_slice_index()
rnn_predictions = compile_predictions_from_phase_output(rnn_preds, slice_index)

In [ ]:
slice_list = cnn_testgen.get_slice_list()
cnn_predictions = compile_predictions_from_slice_output(cnn_preds, slice_list)

In [ ]:
def get_accuracy(preds, labels):
    total_slices = 0
    total_phases = 0
    correct_slices = 0
    correct_phases = 0

    for phase, slices in enumerate(preds):
        total_phases += 1
        correct_phase = True
        for slice, pred in enumerate(slices):
            label = labels[phase][slice]
            if label.sum() == 0:
                pass
            else:
                total_slices += 1
                if label.argmax() == pred.argmax():
                    correct_slices += 1
                else:
                    correct_phase = False
        if correct_phase:
            correct_phases += 1

    slice_accuracy = correct_slices / total_slices
    phase_accuracy = correct_phases / total_phases

    return slice_accuracy, phase_accuracy, total_slices

In [ ]:
# Validate results
y_rnn = np.ndarray((0, 25, 5))
for _, labels in tqdm(rnn_testgen):
    y_rnn = np.concatenate([y_rnn, labels])
get_accuracy(rnn_preds, y_rnn)

In [ ]:
cnn_result = evaluate_predictions(cnn_predictions)
cnn_result

In [ ]:
rnn_result = evaluate_predictions(rnn_predictions)
rnn_result

## Save Results

In [ ]:
result_key = "initial_rnn"
os.makedirs(get_result_dir(result_key), exist_ok=True)

In [ ]:
for key, value in rnn_predictions.items():
    scores = value["scores"]
    for k, v in scores.items():
        scores[k] = str(v)

In [ ]:
with open(get_history_path(result_key), "w") as f:
    json.dump(rnn_history.history, f)
with open(get_predictions_path(result_key), "w") as f:
    json.dump(rnn_predictions, f)
with open(get_result_path(result_key), "w") as f:
    json.dump(rnn_result, f)
rnn_model.save_weights(get_model_weights_path(result_key))

In [ ]:
result_key = "initial_cnn"
os.makedirs(get_result_dir(result_key), exist_ok=True)

In [ ]:
with open(get_history_path(result_key), "w") as f:
    json.dump(cnn_history.history, f)
with open(get_predictions_path(result_key), "w") as f:
    json.dump(cnn_predictions, f)
with open(get_result_path(result_key), "w") as f:
    json.dump(cnn_result, f)
cnn_model.save_weights(get_model_weights_path(result_key))